download transcript

In [ ]:
from playwright.sync_api import sync_playwright
import requests
import os
import time
import glob
import json
import random
def download_transcript_and_audio(url, output_dir="output", title='A'):
    os.makedirs(output_dir, exist_ok=True)
    with sync_playwright() as p:
        browser =  browser = p.chromium.connect_over_cdp("http://localhost:9333")

        # page = browser.new_page()
        contexts = browser.contexts
        page = contexts[0].pages[0]  # 获取第一个 context 的第一个 page
        page.goto(url, timeout=60000)
        # page.goto(url, timeout=60000)
        page.wait_for_load_state("networkidle")  # 等待初次加载完成
        # page.reload()
        # page.wait_for_load_state("networkidle")  # 再次确保加载完
        page.wait_for_selector('div[data-test-id="content-container"]', timeout=10000)

        # 提取 transcript 内容
        transcript_div = page.locator('div[data-test-id="content-container"].T2G6W')
        transcript_text = transcript_div.inner_text()
        with open(f"{output_dir}/{title}.txt", "w", encoding="utf-8") as f:
            f.write(transcript_text)
        print("✅ Transcript 已保存到 transcript.txt")

        # 查找 audio 链接
        # SeekingAlpha 的 transcript 页面通常会有音频 player 标签或链接
        # 判断是否存在 audio 元素
        # 1️⃣ 点击 play-button（如果有）
        # 等待按钮出现并点击
        # page.wait_for_selector('button[data-test-id="play-button"]', timeout=5000)
        # page.wait_for_selector('button[data-test-id="play-button"]', timeout=5000)
        # page.locator('button[data-test-id="play-button"]').first.click()
        # page.locator('button[data-test-id="play-button"]').nth(1).click()
        # buttons = page.locator('button[data-test-id="play-button"]')
        # for i in range(buttons.count()):
        #     btn = buttons.nth(i)
        #     if btn.is_visible():
        #         btn.click()
        #         break
        # time.sleep(2)

        # # 方法一：根据 aria-label 属性
        # download_button = page.query_selector('button[aria-label="download"]')
        # if download_button:
        #     download_button.click()
        #     print("✅ 下载按钮已点击")
        # else:
        #     print("❌ 没有找到下载按钮")

        # time.sleep(30)

        # # 方法一：根据 aria-label 属性 close-modal-button
        # download_button = page.query_selector('button[data-test-id="close-modal-button"]')
        # if download_button:
        #     download_button.click()
        #     print("✅ 关闭按钮已点击")
        # else:
        #     print("❌ 没有找到关闭按钮")


        browser.close()

import re

def sanitize_filename(title):
    # 替换掉文件名中不允许的字符
    return re.sub(r'[<>:"/\\|?*]', '_', title)

should_restart = False  # 全局变量，控制是否重新爬数据

def check_ticker_and_set_flag(ticker):
    global should_restart
    if ticker == 'SCI':
        should_restart = True
        print("✅ 检测到 ticker, 标记为重新开始抓取")

if __name__ == "__main__":
    file_list = glob.glob("/Users/qiqi1996amelia/Downloads/seekingalpha/1/*.json")
    loaded_list = glob.glob("/Users/qiqi1996amelia/Downloads/transcript/*.txt")
    for file in file_list:

        ticker = os.path.splitext(os.path.basename(file))[0].split("_")[-1]
        print(ticker)
        check_ticker_and_set_flag(ticker)
        if should_restart:
            # 假设文件名为 transcripts.json
            with open( file, 'r') as f:
                data = json.load(f)
            
            

            # 提取 link 和 title
            results = [(item['title'], item['link']) for item in data]

            # 输出结果
            for  title, link in results:
                safe_title = sanitize_filename(title)
                matched = any(safe_title in filename for filename in loaded_list)
                if not matched:
                    print(f"Title: {title}")
                    
                    clean_url = link.split('#')[0]
                    if 'Earnings' in title:
                        download_transcript_and_audio( clean_url, f'/Users/qiqi1996amelia/Downloads/transcript/{ticker}', safe_title)
                        delta_y = random.randint(10, 20)
                        time.sleep(delta_y)

In [ ]:
# import os
# import time
# import glob
# import json
# import random
# import re
# import asyncio
# from playwright.async_api import async_playwright

# def sanitize_filename(title):
#     """
#     Sanitizes a string to be a valid filename.
#     """
#     return re.sub(r'[<>:"/\\|?*]', '_', title)

# async def download_transcript(url, output_dir, title):
#     """
#     Uses Playwright to download a transcript from a given URL.
#     This is an asynchronous function.
#     """
#     try:
#         async with async_playwright() as p:
#             browser = await p.chromium.launch()
#             page = await browser.new_page()

#             print(f"Navigating to: {url}")
#             await page.goto(url, timeout=60000)
#             await page.wait_for_load_state("networkidle")

#             error_page_check = page.locator('h1[data-test-id="yikes-page-title"]')
#             if await error_page_check.is_visible():
#                 print(f"❌ Page not found. Skipping download for {title}.")
#                 await browser.close()
#                 return

#             await page.wait_for_selector('div[data-test-id="content-container"]', timeout=10000)

#             transcript_div = page.locator('div[data-test-id="content-container"]')
#             transcript_text = await transcript_div.inner_text()
            
#             os.makedirs(output_dir, exist_ok=True)
            
#             file_path = os.path.join(output_dir, f"{title}.txt")
#             with open(file_path, "w", encoding="utf-8") as f:
#                 f.write(transcript_text)
            
#             print(f"✅ Transcript saved to {file_path}")

#             await browser.close()
#     except Exception as e:
#         print(f"❗ An error occurred while downloading {title}: {e}")

# async def main():
#     """
#     Main asynchronous function to orchestrate the entire process.
#     """
#     # 1. Set your file paths here
#     json_dir = "/Users/mac/Documents/RA/Olivia_Gu/text_analysis/scraper/7/scraper2.0/测试爬取文章/test_json"
#     main_output_dir = "/Users/mac/Documents/RA/Olivia_Gu/text_analysis/scraper/7/scraper2.0/测试爬取文章/transcript"
    
#     # 2. Get the list of JSON files
#     json_files = glob.glob(os.path.join(json_dir, "*.json"))

#     print(f"Found {len(json_files)} JSON files to process.")

#     # 3. Iterate and process each JSON file
#     for file_path in json_files:
#         try:
#             if os.stat(file_path).st_size == 0:
#                 print(f"❗ Skipping empty file: {os.path.basename(file_path)}")
#                 continue

#             file_name = os.path.basename(file_path)
#             ticker = os.path.splitext(file_name)[0].split("_")[-1]

#             ticker_output_dir = os.path.join(main_output_dir, ticker)
#             os.makedirs(ticker_output_dir, exist_ok=True)
            
#             downloaded_files = glob.glob(os.path.join(ticker_output_dir, "*.txt"))
            
#             print(f"\n--- Processing Ticker: {ticker} ---")
            
#             with open(file_path, 'r', encoding='utf-8') as f:
#                 data = json.load(f)
            
#             results = [(item.get('title'), item.get('link')) for item in data]

#             for title, link in results:
#                 if not title or not link:
#                     continue

#                 safe_title = sanitize_filename(title)
                
#                 matched = any(f"{safe_title}.txt" == os.path.basename(f) for f in downloaded_files)

#                 if not matched:
#                     clean_url = link.split('#')[0]
#                     if 'Earnings' in title:
#                         # Call the asynchronous download function
#                         await download_transcript(clean_url, ticker_output_dir, safe_title)
                        
#                         delay = random.uniform(10, 20)
#                         print(f"Pausing for {delay:.2f} seconds...")
#                         await asyncio.sleep(delay)
#                 else:
#                     print(f"✅ Transcript for '{title}' already exists. Skipping.")

#         except json.JSONDecodeError:
#             print(f"❗ Error: {os.path.basename(file_path)} is not a valid JSON file. Skipping.")
#             continue
#         except Exception as e:
#             print(f"❗ An error occurred while processing {file_path}: {e}")

#     print("\n--- All done. ---")

# # Run the asynchronous main function in the notebook
# if __name__ == "__main__":
#     try:
#         import nest_asyncio
#         nest_asyncio.apply()
#     except ImportError:
#         pass
    
#     asyncio.run(main())

Found 18 JSON files to process.

--- Processing Ticker: AACAY ---
Navigating to: https://seekingalpha.com/article/4816115-aac-technologies-holdings-inc-aacay-q2-2025-earnings-call-transcript
❗ An error occurred while downloading AAC Technologies Holdings Inc. (AACAY) Q2 2025 Earnings Call Transcript: Timeout 30000ms exceeded.
Pausing for 12.55 seconds...


KeyboardInterrupt: 

In [ ]:
# import os
# import time
# import glob
# import json
# import random
# import re
# import asyncio
# from playwright.async_api import async_playwright

# def sanitize_filename(title):
#     return re.sub(r'[<>:"/\\|?*]', '_', title)

# async def download_transcript(url, output_dir, title, auth_state_path="auth.json"):
#     """
#     Uses Playwright to download a transcript from a given URL, loading
#     authentication state to access member-only content.
#     """
#     try:
#         async with async_playwright() as p:
#             # 使用 storage_state 参数加载登录状态
#             browser = await p.chromium.launch()
#             context = await browser.new_context(storage_state=auth_state_path)
#             page = await context.new_page()

#             print(f"Navigating to: {url}")
#             await page.goto(url, timeout=60000)
#             await page.wait_for_load_state("networkidle")

#             # 检查页面是否包含“未找到”或“需要登录”的结构
#             # 这是一个更可靠的检查，因为登录状态下不会出现这些结构
#             error_page_check = page.locator('h1[data-test-id="yikes-page-title"]')
#             if await error_page_check.is_visible():
#                 print(f"❌ Page not found. Skipping download for {title}.")
#                 await browser.close()
#                 return

#             # 等待包含文字记录内容的主 div 元素加载完成
#             # 如果加载了登录状态，这个选择器应该能找到内容
#             await page.wait_for_selector('div[data-test-id="content-container"]', timeout=10000)

#             transcript_div = page.locator('div[data-test-id="content-container"]')
#             transcript_text = await transcript_div.inner_text()
            
#             os.makedirs(output_dir, exist_ok=True)
            
#             file_path = os.path.join(output_dir, f"{title}.txt")
#             with open(file_path, "w", encoding="utf-8") as f:
#                 f.write(transcript_text)
            
#             print(f"✅ Transcript saved to {file_path}")

#             await browser.close()
#     except Exception as e:
#         print(f"❗ An error occurred while downloading {title}: {e}")

# async def main():
#     json_dir = "/Users/mac/Documents/RA/Olivia_Gu/text_analysis/scraper/7/scraper2.0/测试爬取文章/test_json"
#     main_output_dir = "/Users/mac/Documents/RA/Olivia_Gu/text_analysis/scraper/7/scraper2.0/测试爬取文章/transcript"
#     auth_state_path = "auth.json"

#     # 在运行前，确保你已经运行了保存登录状态的脚本，并且 auth.json 文件存在
#     if not os.path.exists(auth_state_path):
#         print("❗ 登录状态文件 'auth.json' 不存在。请先运行 save_auth_state.py 来生成它。")
#         return
    
#     json_files = glob.glob(os.path.join(json_dir, "*.json"))

#     print(f"Found {len(json_files)} JSON files to process.")

#     for file_path in json_files:
#         try:
#             if os.stat(file_path).st_size == 0:
#                 print(f"❗ Skipping empty file: {os.path.basename(file_path)}")
#                 continue

#             file_name = os.path.basename(file_path)
#             ticker = os.path.splitext(file_name)[0].split("_")[-1]

#             ticker_output_dir = os.path.join(main_output_dir, ticker)
#             os.makedirs(ticker_output_dir, exist_ok=True)
            
#             downloaded_files = glob.glob(os.path.join(ticker_output_dir, "*.txt"))
            
#             print(f"\n--- Processing Ticker: {ticker} ---")
            
#             with open(file_path, 'r', encoding='utf-8') as f:
#                 data = json.load(f)
            
#             results = [(item.get('title'), item.get('link')) for item in data]

#             for title, link in results:
#                 if not title or not link:
#                     continue

#                 safe_title = sanitize_filename(title)
                
#                 matched = any(f"{safe_title}.txt" == os.path.basename(f) for f in downloaded_files)

#                 if not matched:
#                     clean_url = link.split('#')[0]
#                     if 'Earnings' in title:
#                         await download_transcript(clean_url, ticker_output_dir, safe_title, auth_state_path)
                        
#                         delay = random.uniform(10, 20)
#                         print(f"Pausing for {delay:.2f} seconds...")
#                         await asyncio.sleep(delay)
#                 else:
#                     print(f"✅ Transcript for '{title}' already exists. Skipping.")

#         except json.JSONDecodeError:
#             print(f"❗ Error: {os.path.basename(file_path)} is not a valid JSON file. Skipping.")
#             continue
#         except Exception as e:
#             print(f"❗ An error occurred while processing {file_path}: {e}")

#     print("\n--- All done. ---")

# if __name__ == "__main__":
#     try:
#         import nest_asyncio
#         nest_asyncio.apply()
#     except ImportError:
#         pass
    
#     asyncio.run(main())

In [9]:
import os
import time
import glob
import json
import random
import re
import asyncio
from playwright.async_api import async_playwright

def sanitize_filename(title):
    return re.sub(r'[<>:"/\\|?*]', '_', title)

async def download_transcript(url, output_dir, title):
    """
    Uses Playwright to download a transcript from a given URL by connecting
    to a live browser instance.
    """
    try:
        async with async_playwright() as p:
            # 修改这里，连接到已经运行的浏览器实例
            # 注意：这里不再需要传入 auth_state_path
            browser = await p.chromium.connect_over_cdp("http://localhost:9222")

            # Playwright会使用当前连接的浏览器的第一个context和page
            contexts = browser.contexts
            if not contexts:
                print("❗ No active browser context found. Please ensure the browser is open.")
                return

            page = contexts[0].pages[0]

            print(f"Navigating to: {url}")
            await page.goto(url, timeout=60000)
            await page.wait_for_load_state("networkidle")

            # 检查页面是否包含“未找到”或“需要登录”的结构
            # 登录后，这些结构通常不会出现
            error_page_check = page.locator('h1[data-test-id="yikes-page-title"]')
            if await error_page_check.is_visible():
                print(f"❌ Page not found. Skipping download for {title}.")
                return

            # 等待包含文字记录内容的主 div 元素加载完成
            await page.wait_for_selector('div[data-test-id="content-container"]', timeout=10000)

            transcript_div = page.locator('div[data-test-id="content-container"]')
            transcript_text = await transcript_div.inner_text()
            
            os.makedirs(output_dir, exist_ok=True)
            
            file_path = os.path.join(output_dir, f"{title}.txt")
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(transcript_text)
            
            print(f"✅ Transcript saved to {file_path}")

    except Exception as e:
        print(f"❗ An error occurred while downloading {title}: {e}")

async def main():
    json_dir = "/Users/mac/Documents/RA/Olivia_Gu/text_analysis/scraper/7/scraper2.0/测试爬取文章/test_json"
    main_output_dir = "/Users/mac/Documents/RA/Olivia_Gu/text_analysis/scraper/7/scraper2.0/测试爬取文章/transcript"
    
    json_files = glob.glob(os.path.join(json_dir, "*.json"))

    print(f"Found {len(json_files)} JSON files to process.")

    for file_path in json_files:
        try:
            if os.stat(file_path).st_size == 0:
                print(f"❗ Skipping empty file: {os.path.basename(file_path)}")
                continue

            file_name = os.path.basename(file_path)
            ticker = os.path.splitext(file_name)[0].split("_")[-1]

            ticker_output_dir = os.path.join(main_output_dir, ticker)
            os.makedirs(ticker_output_dir, exist_ok=True)
            
            downloaded_files = glob.glob(os.path.join(ticker_output_dir, "*.txt"))
            
            print(f"\n--- Processing Ticker: {ticker} ---")
            
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            results = [(item.get('title'), item.get('link')) for item in data]

            for title, link in results:
                if not title or not link:
                    continue

                safe_title = sanitize_filename(title)
                
                matched = any(f"{safe_title}.txt" == os.path.basename(f) for f in downloaded_files)

                if not matched:
                    clean_url = link.split('#')[0]
                    if 'Earnings' in title:
                        await download_transcript(clean_url, ticker_output_dir, safe_title)
                        
                        delay = random.uniform(10, 20)
                        print(f"Pausing for {delay:.2f} seconds...")
                        await asyncio.sleep(delay)
                else:
                    print(f"✅ Transcript for '{title}' already exists. Skipping.")

        except json.JSONDecodeError:
            print(f"❗ Error: {os.path.basename(file_path)} is not a valid JSON file. Skipping.")
            continue
        except Exception as e:
            print(f"❗ An error occurred while processing {file_path}: {e}")

    print("\n--- All done. ---")

if __name__ == "__main__":
    try:
        import nest_asyncio
        nest_asyncio.apply()
    except ImportError:
        pass
    
    asyncio.run(main())

Found 18 JSON files to process.

--- Processing Ticker: AACAY ---
Navigating to: https://seekingalpha.com/article/4816115-aac-technologies-holdings-inc-aacay-q2-2025-earnings-call-transcript


Task exception was never retrieved
future: <Task finished name='Task-18' coro=<Waiter.reject_on_timeout.<locals>.reject() done, defined at /Users/mac/anaconda3/lib/python3.11/site-packages/playwright/_impl/_waiter.py:85> exception=TargetClosedError(': Target page, context or browser has been closed')>
Traceback (most recent call last):
  File "/Users/mac/anaconda3/lib/python3.11/asyncio/tasks.py", line 267, in __step
    result = coro.send(None)
             ^^^^^^^^^^^^^^^
  File "/Users/mac/anaconda3/lib/python3.11/site-packages/playwright/_impl/_waiter.py", line 87, in reject
    self._reject(TimeoutError(message))
  File "/Users/mac/anaconda3/lib/python3.11/site-packages/playwright/_impl/_waiter.py", line 111, in _reject
    self._wait_for_event_info_after(self._wait_id, exception)
  File "/Users/mac/anaconda3/lib/python3.11/site-packages/playwright/_impl/_waiter.py", line 52, in _wait_for_event_info_after
    self._channel._connection.wrap_api_call_sync(
  File "/Users/mac/anacond

✅ Transcript saved to /Users/mac/Documents/RA/Olivia_Gu/text_analysis/scraper/7/scraper2.0/测试爬取文章/transcript/AACAY/AAC Technologies Holdings Inc. (AACAY) Q2 2025 Earnings Call Transcript.txt
Pausing for 16.13 seconds...
✅ Transcript for 'AAC Technologies Holdings Inc. (AACAY) Q2 2023 Earnings Call Transcript' already exists. Skipping.
Navigating to: https://seekingalpha.com/article/4536825-aac-technologies-holdings-inc-aacay-management-on-q2-2022-results-earnings-call-transcript
✅ Transcript saved to /Users/mac/Documents/RA/Olivia_Gu/text_analysis/scraper/7/scraper2.0/测试爬取文章/transcript/AACAY/AAC Technologies Holdings Inc. (AACAY) Management on Q2 2022 Results - Earnings Call Transcript.txt
Pausing for 10.79 seconds...
Navigating to: https://seekingalpha.com/article/4536695-aac-technologies-holdings-inc-2022-q2-results-earnings-call-presentation


KeyboardInterrupt: 

In [ ]:
import os
import time
import glob
import json
import random
import re
import asyncio
from playwright.async_api import async_playwright
from playwright.async_api._generated import Page

def sanitize_filename(title):
    """
    Sanitizes a string to be a valid filename by replacing invalid characters.
    """
    return re.sub(r'[<>:"/\\|?*]', '_', title)

async def wait_for_captcha_and_resume(page: Page):
    """
    Checks for a captcha page and pauses the script for manual resolution.
    """
    print("🤖 Checking for CAPTCHA...")
    try:
        captcha_button = page.get_by_text("按住").first
        await captcha_button.wait_for(state="visible", timeout=5000)
        
        print("\n" + "="*50)
        print("⚠️ CAPTCHA detected! Please manually solve it in the browser.")
        print("💡 Once done, return to the console and press Enter to continue.")
        print("="*50 + "\n")
        
        await asyncio.to_thread(input, "Press Enter to continue...")
        
        await page.wait_for_load_state('domcontentloaded')
        print("✅ CAPTCHA solved, resuming execution.")
        
    except Exception:
        print("🚀 No CAPTCHA detected, continuing.")
        pass

async def download_transcript(page: Page, url: str, output_dir: str, title: str):
    """
    Downloads a transcript using an existing Playwright page.
    Returns True if successful, a tuple (False, error_message) otherwise.
    """
    try:
        print(f"Navigating to: {url}")
        
        await page.goto(url, timeout=60000)
        
        await wait_for_captcha_and_resume(page)
        
        await page.wait_for_load_state("networkidle")

        # Use a more specific locator to avoid "strict mode" errors
        transcript_locator = 'div.T2G6W[data-test-id="content-container"]'
        
        error_page_check = page.locator('h1[data-test-id="yikes-page-title"]')
        if await error_page_check.is_visible():
            raise Exception("Page not found on Seeking Alpha.")
        
        await page.wait_for_selector(transcript_locator, timeout=10000)
        
        transcript_div = page.locator(transcript_locator)
        transcript_text = await transcript_div.inner_text()
        
        os.makedirs(output_dir, exist_ok=True)
        
        file_path = os.path.join(output_dir, f"{title}.txt")
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(transcript_text)
        
        print(f"✅ Transcript saved to {file_path}")
        return True

    except Exception as e:
        # Catch any exception and return it as a failure
        return False, str(e)

async def main():
    json_dir = "/Users/mac/Documents/RA/Olivia_Gu/text_analysis/scraper/7/scraper2.0/测试爬取文章/test_json"
    main_output_dir = "/Users/mac/Documents/RA/Olivia_Gu/text_analysis/scraper/7/scraper2.0/测试爬取文章/transcript"
    
    json_files = glob.glob(os.path.join(json_dir, "*.json"))
    print(f"Found {len(json_files)} JSON files to process.")

    async with async_playwright() as p:
        try:
            browser = await p.chromium.connect_over_cdp("http://localhost:9222")
            context = browser.contexts[0]
            page = context.pages[0]
        except Exception as e:
            print(f"❗ Failed to connect to browser instance: {e}")
            print("Please ensure you have run '.../Google Chrome --remote-debugging-port=9222 ...' in the terminal.")
            return

        for file_path in json_files:
            try:
                if os.stat(file_path).st_size == 0:
                    print(f"❗ Skipping empty file: {os.path.basename(file_path)}")
                    continue

                file_name = os.path.basename(file_path)
                ticker = os.path.splitext(file_name)[0].split("_")[-1]

                ticker_output_dir = os.path.join(main_output_dir, ticker)
                os.makedirs(ticker_output_dir, exist_ok=True)
                
                # 定义每个公司的失败日志文件路径
                failed_links_file = os.path.join(ticker_output_dir, "failed.json")
                failed_links = []
                
                downloaded_files = glob.glob(os.path.join(ticker_output_dir, "*.txt"))
                
                print(f"\n--- Processing Ticker: {ticker} ---")
                
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                
                results = [(item.get('title'), item.get('link')) for item in data]

                for title, link in results:
                    if not title or not link:
                        continue

                    safe_title = sanitize_filename(title)
                    
                    # 检查是否已下载
                    if any(f"{safe_title}.txt" == os.path.basename(f) for f in downloaded_files):
                        print(f"✅ Transcript for '{title}' already exists. Skipping.")
                        continue
                    
                    clean_url = link.split('#')[0]
                    if 'Earnings' in title:
                        download_result = await download_transcript(page, clean_url, ticker_output_dir, safe_title)
                        
                        # 检查下载结果，如果失败则添加到列表中
                        if isinstance(download_result, tuple):
                            _, error_message = download_result
                            failed_links.append({
                                "title": title,
                                "link": clean_url,
                                "error": error_message
                            })
                        
                        delay = random.uniform(2, 5)
                        print(f"Pausing for {delay:.2f} seconds...")
                        await asyncio.sleep(delay)
                
                # 循环结束后，将该公司的失败链接保存到其专属的 failed.json 文件
                if failed_links:
                    with open(failed_links_file, 'w', encoding='utf-8') as f:
                        json.dump(failed_links, f, indent=4, ensure_ascii=False)
                    print(f"❗ {len(failed_links)} failed links saved to {failed_links_file}")

            except json.JSONDecodeError:
                print(f"❗ Error: {os.path.basename(file_path)} is not a valid JSON file. Skipping.")
                continue
            except Exception as e:
                print(f"❗ An unexpected error occurred while processing {file_path}: {e}")

    print("\n--- All done. ---")
    

In [14]:
if __name__ == "__main__":
    try:
        import nest_asyncio
        nest_asyncio.apply()
    except ImportError:
        pass
    
    asyncio.run(main())

Found 12 JSON files to process.

--- Processing Ticker: AACAY ---
Navigating to: https://seekingalpha.com/article/4816115-aac-technologies-holdings-inc-aacay-q2-2025-earnings-call-transcript
🤖 Checking for CAPTCHA...
🚀 No CAPTCHA detected, continuing.
✅ Transcript saved to /Users/mac/Documents/RA/Olivia_Gu/text_analysis/scraper/7/scraper2.0/测试爬取文章/transcript/AACAY/AAC Technologies Holdings Inc. (AACAY) Q2 2025 Earnings Call Transcript.txt
Pausing for 2.70 seconds...
Navigating to: https://seekingalpha.com/article/4632706-aac-technologies-holdings-inc-aacay-q2-2023-earnings-call-transcript
🤖 Checking for CAPTCHA...
🚀 No CAPTCHA detected, continuing.
✅ Transcript saved to /Users/mac/Documents/RA/Olivia_Gu/text_analysis/scraper/7/scraper2.0/测试爬取文章/transcript/AACAY/AAC Technologies Holdings Inc. (AACAY) Q2 2023 Earnings Call Transcript.txt
Pausing for 4.42 seconds...
Navigating to: https://seekingalpha.com/article/4536825-aac-technologies-holdings-inc-aacay-management-on-q2-2022-results-ea